In [70]:
import requests
import pandas as pd
import os

In [71]:
url = r'http://127.0.0.1:5000/country'
req = requests.get(url)
print(f"Status da requisição: {req.status_code}")

Status da requisição: 200


In [72]:
# Vamos garantir através do status code que a requisição foi bem sucedida
status = req.status_code

if status == 200:  
    data = req.json()
    df_country = pd.DataFrame(data)
else:
    print(f"Erro na requisição: {status}")

# Colunas relevantes: Nome, Código da Moeda(currency), População, Capital, Código(Continente?) e Área
# Transpondo o dataframe para poder fazer um merge baseado na coluna "code"
df_country = df_country.T.reset_index()

# Definir nomes das colunas
df_country.columns = ['code', 'name', 'currency', 'population', 'capital', 'continente', 'area']

df_country


,code,name,currency,population,capital,continente,area
0,AND,Andorra,EUR,77006,Andorra la Vella,Europe,468.0
1,ARE,United Arab Emirates,AED,9630959,Abu Dhabi,Asia,82880.0
2,AFG,Afghanistan,AFN,37172386,Kabul,Asia,647500.0
3,ATG,Antigua and Barbuda,XCD,96286,St John's,North America,443.0
4,AIA,Anguilla,XCD,13254,The Valley,North America,102.0
...,...,...,...,...,...,...,...
245,YEM,Yemen,YER,28498687,Sanaa,Asia,527970.0
246,MYT,Mayotte,EUR,279471,Mamoudzou,Africa,374.0
247,ZAF,South Africa,ZAR,57779622,Pretoria,Africa,1219912.0
248,ZMB,Zambia,ZMW,17351822,Lusaka,Africa,752614.0


In [73]:

# df = pd.read_csv('./gdp_data.csv', sep=',') -- caminho relativo não está funcionando, não entendi o motivo.

caminho_arquivo = os.path.abspath('./puc/coleta-e-preparacao-dados/Coleta-e-preparacao-dados/gdp_data.csv') # usando caminho absoluto
df_gdp = pd.read_csv(caminho_arquivo, sep=',')
print(df_gdp.head())

                  Country Name Country Code     Indicator Name  \
0                        Aruba          ABW  GDP (current US$)   
1  Africa Eastern and Southern          AFE  GDP (current US$)   
2                  Afghanistan          AFG  GDP (current US$)   
3   Africa Western and Central          AFW  GDP (current US$)   
4                       Angola          AGO  GDP (current US$)   

   Indicator Code          1960          1961          1962          1963  \
0  NY.GDP.MKTP.CD           NaN           NaN           NaN           NaN   
1  NY.GDP.MKTP.CD  2.129081e+10  2.180870e+10  2.370727e+10  2.821034e+10   
2  NY.GDP.MKTP.CD  5.377778e+08  5.488889e+08  5.466667e+08  7.511112e+08   
3  NY.GDP.MKTP.CD  1.040414e+10  1.112789e+10  1.194319e+10  1.267633e+10   
4  NY.GDP.MKTP.CD           NaN           NaN           NaN           NaN   

           1964          1965  ...          2013          2014          2015  \
0           NaN           NaN  ...  2.727933e+09  2.791061e+

In [74]:
# Verificando quantos dados do tipo n.a existem
df_gdp.isna().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              133
                 ... 
2018                9
2019               11
2020               14
2021               21
Unnamed: 66       266
Length: 67, dtype: int64

In [75]:
# Renomeando a coluna para que fique igual ao do DataFrame "df_country" e terem uma coluna em comum.
df_gdp.rename(columns={'Country Code': 'code'}, inplace=True)
df_final = df_country.merge(df_gdp, on='code', how='left')
df_final.head()

,code,name,currency,population,capital,continente,area,Country Name,Indicator Name,Indicator Code,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,AND,Andorra,EUR,77006,Andorra la Vella,Europe,468.0,Andorra,GDP (current US$),NY.GDP.MKTP.CD,...,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.155065e+09,2.891022e+09,3.330282e+09,NaN
1,ARE,United Arab Emirates,AED,9630959,Abu Dhabi,Asia,82880.0,United Arab Emirates,GDP (current US$),NY.GDP.MKTP.CD,...,4.002185e+11,4.141054e+11,3.702755e+11,3.692553e+11,3.905168e+11,4.270494e+11,4.179897e+11,3.494730e+11,4.150216e+11,NaN
2,AFG,Afghanistan,AFN,37172386,Kabul,Asia,647500.0,Afghanistan,GDP (current US$),NY.GDP.MKTP.CD,...,2.056449e+10,2.055058e+10,1.999816e+10,1.801956e+10,1.889635e+10,1.841885e+10,1.890449e+10,2.014344e+10,1.478686e+10,NaN
3,ATG,Antigua and Barbuda,XCD,96286,St John's,North America,443.0,Antigua and Barbuda,GDP (current US$),NY.GDP.MKTP.CD,...,1.181448e+09,1.249733e+09,1.336693e+09,1.436585e+09,1.467978e+09,1.605944e+09,1.687533e+09,1.370281e+09,1.471126e+09,NaN
4,AIA,Anguilla,XCD,13254,The Valley,North America,102.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
print(df_final.columns)
# Colunas pedidas pelo projeto: “codigo”, “nome”, “populacao”, “capital”, “area”, “continente” e "PIB"
# Então, precisarei reorganizar as colunas.

Index(['code', 'name', 'currency', 'population', 'capital', 'continente',
       'area', 'Country Name', 'Indicator Name', 'Indicator Code', '1960',
       '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Unnamed: 66'],
      dtype='object')


In [77]:
# Renomeando as colunas para deixar mais organizado o csv final
df_final.rename(columns={
    'code':       'codigo',
    'name':       'nome',
    'population': 'populacao',
    'capital':    'capital',
    'area':       'area',
    'continente': 'continente'
}, inplace=True)
print(df_final.columns)

Index(['codigo', 'nome', 'currency', 'populacao', 'capital', 'continente',
       'area', 'Country Name', 'Indicator Name', 'Indicator Code', '1960',
       '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Unnamed: 66'],
      dtype='object')


In [78]:
colunas_anos = [str(ano) for ano in range(1960, 2022)]
# print(colunas_anos) ['1960', '1961', '1962', '1963', '1964', '1965', '1966' .... '2021']
df_final = df_final[['codigo', 'nome', 'populacao', 'capital', 'area', 'continente'] + colunas_anos]

print(df_final.columns)


Index(['codigo', 'nome', 'populacao', 'capital', 'area', 'continente', '1960',
       '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969',
       '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978',
       '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')


In [79]:
# EXPORTANDO PARA UM CSV
df_final.to_csv('./puc/coleta-e-preparacao-dados/Coleta-e-preparacao-dados/fase1.csv', index=False, encoding='utf-8')

